## **Loading Libraries**

In [267]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

## **Loading Dataset**

In [268]:
df=pd.read_excel("Data/transactions.xlsx")
df.head()

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One)
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items
3,1000692089,2024-04-05 09:39:58,Scott Lopez,5,14.64,Mobile Payment,Atlanta,Pharmacy,False,Homemaker,Summer,Discount on Selected Items
4,1000328702,2021-05-28 04:16:54,Crystal Adams,4,62.27,Credit Card,Miami,Convenience Store,False,Retiree,Summer,NaN


## **Data Pre-Processing**

---

In [269]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38500 entries, 0 to 38499
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Transaction_ID     38500 non-null  int64  
 1   Date               38500 non-null  object 
 2   Customer_Name      38500 non-null  object 
 3   Total_Items        38500 non-null  int64  
 4   Amount($)          38500 non-null  float64
 5   Payment_Method     38500 non-null  object 
 6   City               38500 non-null  object 
 7   Store_Type         38500 non-null  object 
 8   Discount_Applied   38500 non-null  bool   
 9   Customer_Category  38500 non-null  object 
 10  Season             38500 non-null  object 
 11  Promotion          25529 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(8)
memory usage: 3.3+ MB


- The dataset contains 38,500 entries across 12 columns, with the `Promotion` column having 25,529 non-null entries, indicating missing values. All other columns are complete. 
- The `Date` column is currently of type `object`, which should be converted to `datetime` for proper date handling. 

---

### Handling the Date Data Type

In [270]:
df["Date"]=pd.to_datetime(df.Date) # Converting 'Date' column to datetime dtype

In [271]:
df["Date"].dtype ##Cheking the dtype

dtype('<M8[ns]')

In [272]:
str(df["Date"].dtype)  ##Cheking the dtype

'datetime64[ns]'

In [273]:
df.head(3)

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One)
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items


---

### Checking Missing Values 

In [274]:
df.isnull().sum()

Transaction_ID           0
Date                     0
Customer_Name            0
Total_Items              0
Amount($)                0
Payment_Method           0
City                     0
Store_Type               0
Discount_Applied         0
Customer_Category        0
Season                   0
Promotion            12971
dtype: int64

The `Promotion` column has 12,971 missing values, which accounts for a significant portion of the dataset. Further analysis is needed to determine how to handle these missing values.

---

### Checking the "Promotion" Column

In [275]:
df["Promotion"].value_counts()

Promotion
Discount on Selected Items    12811
BOGO (Buy One Get One)        12718
Name: count, dtype: int64

Although the `Promotion` column contains missing values (12,971 in total), it is observed that the non-null entries consist primarily of two distinct promotion types: **"Discount on Selected Items"** (12,811 occurrences) and **"BOGO (Buy One Get One)"** (12,718 occurrences). Since there is no explicit "None" or "No Promotion" value in the dataset, it is likely that the missing values represent transactions that did not have a promotional offer applied. Therefore, further treatment of these missing values may involve interpreting them as transactions with no promotion.

---

### Handling Missing Values

In [276]:
df["Promotion"].fillna("No Promotion", inplace=True) ## Filling The Missing Values With No Promotion

In [277]:
df.isnull().sum() ##Checking The Missing Values Again

Transaction_ID       0
Date                 0
Customer_Name        0
Total_Items          0
Amount($)            0
Payment_Method       0
City                 0
Store_Type           0
Discount_Applied     0
Customer_Category    0
Season               0
Promotion            0
dtype: int64

In [278]:
df["Promotion"].value_counts() ##Verify

Promotion
No Promotion                  12971
Discount on Selected Items    12811
BOGO (Buy One Get One)        12718
Name: count, dtype: int64

The missing values in the `Promotion` column were filled with the label **"No Promotion"**. This effectively indicates that transactions without a promotional offer are categorized as "No Promotion", ensuring a complete and consistent dataset for further analysis.

---

## Checking For Duplicate Values

In [279]:
num_duplicates = df.duplicated().sum()
print(f"Total number of duplicate rows in the dataset: {num_duplicates}")


Total number of duplicate rows in the dataset: 0


---

### Summary Statistics of the Dataset

In [280]:
df.describe()

,Transaction_ID,Date,Total_Items,Amount($)
count,3.850000e+04,38500,38500.000000,38500.000000
mean,1.000500e+09,2022-03-10 05:20:27.853402624,5.490649,52.459843
min,1.000000e+09,2020-01-01 03:51:50,1.000000,5.000000
25%,1.000248e+09,2021-02-02 04:45:56.500000,3.000000,28.760000
50%,1.000501e+09,2022-03-11 09:14:40.500000,5.000000,52.260000
75%,1.000751e+09,2023-04-13 13:47:41.750000128,8.000000,76.350000
max,1.001000e+09,2024-05-18 19:06:29,10.000000,100.000000
std,2.890708e+05,NaN,2.868476,27.442214


The statistical summary of the dataset shows that for the numerical columns, the mean and 50th percentile (median) values are closely aligned, indicating a relatively symmetric distribution with minimal skew. The values across different columns, including `Transaction_ID`, `Total_Items`, and `Amount($)`, suggest a consistent range without significant deviations, implying the absence of notable outliers in the dataset.

---

## **Data Preprocessing Completed** ☑️☑️

The data preprocessing phase has been successfully completed, with missing values handled and data types appropriately assigned. After reviewing the dataset, no significant outliers or data duplicates were found. With the dataset now cleaned and ready, we will proceed to the next phase of the analysis, where we will dive deeper into exploring and interpreting the data to derive actionable insights.

--- 

# **Finding meaningful insights**

### 1. What is the average transaction amount ($) across different store types, and how does it vary by season?

In [281]:
df.head(3)

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One)
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items


In [282]:
df_aggregated = df.groupby(["Store_Type", "Season"])["Amount($)"].mean().reset_index()
df_aggregated


,Store_Type,Season,Amount($)
0,Convenience Store,Fall,52.872504
1,Convenience Store,Spring,53.537230
2,Convenience Store,Summer,53.345305
3,Convenience Store,Winter,51.595034
4,Department Store,Fall,51.384757
5,Department Store,Spring,52.573781
6,Department Store,Summer,52.782716
7,Department Store,Winter,52.575652
8,Pharmacy,Fall,52.363508
9,Pharmacy,Spring,52.458672


In [283]:
fig = px.bar(
    df_aggregated,
    x='Store_Type',
    y='Amount($)',
    color='Season',
    title='Average Transaction Amount by Store Type and Season',
    labels={'Amount($)': 'Average Amount ($)', 'Store_Type': 'Store Type'},
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set3  # Consistent professional palette
)

# Customize layout
fig.update_layout(
    title={
        'text': 'Average Transaction Amount by Store Type and Season',
        'x': 0.5,  # Center alignment
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'family': 'Arial', 'color': '#333'}
    },
    xaxis={
        'title': {'text': 'Store Type', 'font': {'size': 16, 'color': '#555'}},
        'tickfont': {'size': 12, 'color': '#555'}
    },
    yaxis={
        'title': {'text': 'Average Amount ($)', 'font': {'size': 16, 'color': '#555'}},
        'tickfont': {'size': 12, 'color': '#555'},
        'gridcolor': 'rgba(200, 200, 200, 0.3)'  # Subtle gridlines
    },
    legend={
        'title': {'text': 'Season', 'font': {'size': 14, 'color': '#555'}},
        'font': {'size': 12, 'color': '#555'},
        'bgcolor': 'rgba(255, 255, 255, 0.8)',
        'bordercolor': '#ccc',
        'borderwidth': 1
    },
    plot_bgcolor='rgba(245, 245, 245, 1)',  # Light background
    paper_bgcolor='white',
)


fig.show()


In [284]:
fig = px.bar(
    df_aggregated,
    x='Store_Type',
    y='Amount($)',
    color='Season',
    title='Average Transaction Amount by Store Type and Season',
    labels={'Amount($)': 'Average Amount ($)', 'Store_Type': 'Store Type'},
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set3  # Professional color palette
)

# Customize layout
fig.update_layout(
    title={
        'text': 'Average Transaction Amount by Store Type and Season',
        'x': 0.5,  # Center alignment
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'family': 'Arial', 'color': '#333'}
    },
    xaxis={
        'title': {'text': 'Store Type', 'font': {'size': 16, 'color': '#555'}},
        'tickfont': {'size': 12, 'color': '#555'}
    },
    yaxis={
        'title': {'text': 'Average Amount ($)', 'font': {'size': 16, 'color': '#555'}},
        'tickfont': {'size': 12, 'color': '#555'},
        'gridcolor': 'rgba(200, 200, 200, 0.3)'  # Subtle gridlines
    },
    legend={
        'title': {'text': 'Season', 'font': {'size': 14, 'color': '#555'}},
        'font': {'size': 12, 'color': '#555'},
        'bgcolor': 'rgba(255, 255, 255, 0.8)',
        'bordercolor': '#ccc',
        'borderwidth': 1
    },
    plot_bgcolor='rgba(245, 245, 245, 1)', 
    paper_bgcolor='white'
)

# Add borders to bars and values inside the bars
fig.update_traces(
    marker_line_color='white',  
    marker_line_width=1,  
    texttemplate='%{y:.2f}',  # Format to show values inside bars
    textposition='inside'  # Position values inside the bars
)

fig.show()


In [285]:
import plotly.express as px

# Create the bar plot
fig = px.bar(
    df_aggregated,
    x='Store_Type',
    y='Amount($)',
    color='Season',
    title='Average Transaction Amount by Store Type and Season',
    labels={'Amount($)': 'Average Amount ($)', 'Store_Type': 'Store Type'},
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set3  # Professional color palette (Set1)
)

# Customize layout to follow the exact theme
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1300,
    plot_bgcolor='white',  # Set white background for the plot
    paper_bgcolor='white',  # White paper background
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='Season',
        font=dict(size=13, family='Arial'),
        bordercolor='black',
        borderwidth=1,
        bgcolor='rgba(255, 255, 255, 0.7)',  # Semi-transparent background for the legend
    ),
    xaxis=dict(
        title='Store Type',
        tickangle=-45,  # Rotate x-axis labels for readability
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,  # Show gridlines for better readability
        gridcolor='lightgray',  # Light gray gridlines
    ),
    yaxis=dict(
        title='Average Amount ($)',
        showgrid=True,  # Show gridlines for better readability
        gridcolor='lightgray',  # Light gray gridlines
    )
)

# Enhance text visibility and bar appearance
fig.update_traces(
    texttemplate='%{y:.2f}',  # Format to show values inside bars
    textposition='inside',  # Place values inside bars for visibility
    marker_line_width=1.5,  # Add borders around the bars for definition
    textfont=dict(size=11, color='black', family='Arial', weight='bold')  # Text styling
)

# Show the plot
fig.show()


### 2. Which payment method is most commonly used in high-value transactions (above the average transaction amount), and how does it differ across cities?

In [286]:
df.head(3)

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One)
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items


In [287]:
# Calculate the average transaction amount
average_transaction_amount = df['Amount($)'].mean()

# Filter for high-value transactions
high_value_transactions = df[df['Amount($)'] > average_transaction_amount]

# Count the payment methods used in high-value transactions across cities
payment_method_counts = high_value_transactions.groupby(['City', 'Payment_Method']).size().reset_index(name='Count')

# Identify the most commonly used payment method in each city
most_common_payment_method = payment_method_counts.loc[payment_method_counts.groupby('City')['Count'].idxmax()]

In [288]:
payment_method_counts

,City,Payment_Method,Count
0,Atlanta,Cash,467
1,Atlanta,Credit Card,452
2,Atlanta,Debit Card,451
3,Atlanta,Mobile Payment,506
4,Boston,Cash,485
5,Boston,Credit Card,471
6,Boston,Debit Card,484
7,Boston,Mobile Payment,427
8,Chicago,Cash,516
9,Chicago,Credit Card,507


In [289]:
most_common_payment_method

,City,Payment_Method,Count
3,Atlanta,Mobile Payment,506
4,Boston,Cash,485
8,Chicago,Cash,516
14,Dallas,Debit Card,538
16,Houston,Cash,492
22,Los Angeles,Debit Card,518
25,Miami,Credit Card,489
31,New York,Mobile Payment,510
33,San Francisco,Credit Card,504
38,Seattle,Debit Card,512


In [290]:
fig = px.bar(
    payment_method_counts,
    x='City',
    y='Count',
    color='Payment_Method',  # Group bars by payment method
    barmode='group',  # Side-by-side bars
    title='<b>Most Common Payment Method in High-Value Transactions by City</b>',
    labels={'Count': 'Number of Transactions', 'City': 'City'},
    color_discrete_sequence=px.colors.qualitative.Set1,  # Professional color palette
    text='Count'  # Display the count of transactions on top of the bars
)

# Update layout to improve readability and presentation
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1300,
    plot_bgcolor='white',
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='Payment Method',
    
    ),
    xaxis=dict(
        title='City',
        tickangle=-45,  # Rotate x-axis labels for readability
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        title='Number of Transactions',
        showgrid=True,
        gridcolor='lightgray',
    )
)

# Enhance text visibility and bar appearance
fig.update_traces(
    texttemplate='%{text}',  # Show the actual transaction count
    textposition='outside',  # Place text outside bars for visibility
    marker_line_width=1.5,  # Add borders around the bars for definition
    textfont=dict(size=11, color='black', family='Arial', weight='bold')
)

# Show the plot
fig.show()


### 3. How do the sales amounts in transactions with discounts compare to those without discounts, and what trends can be observed over the month?

In [291]:
df.head(3)

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One)
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items


In [292]:
# Extract only the month name (e.g., 'January', 'February', etc.)
df['Month'] = df['Date'].dt.strftime('%B')

# Group by Month and Discount status, summing the sales amounts
sales_comparison = df.groupby(['Month', 'Discount_Applied'])['Amount($)'].sum().reset_index()

# Sort by month order (using a predefined month order list)
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
sales_comparison['Month'] = pd.Categorical(sales_comparison['Month'], categories=month_order, ordered=True)

# Sort the data by month for chronological order
sales_comparison = sales_comparison.sort_values(by='Month').reset_index(drop=True)

# Display the result
sales_comparison


,Month,Discount_Applied,Amount($)
0,January,False,94368.87
1,January,True,99399.06
2,February,False,88972.20
3,February,True,89528.82
4,March,True,104664.82
5,March,False,101094.87
6,April,False,91498.56
7,April,True,94579.38
8,May,True,88251.50
9,May,False,87319.30


In [293]:
fig = px.line(sales_comparison, x='Month', y='Amount($)', color='Discount_Applied',
              title='Sales Amounts: With vs Without Discounts Over the Month',
              labels={'Amount($)': 'Total Sales Amount ($)', 'Month': 'Month'},
              markers=True)
fig.show()

In [294]:
import plotly.express as px

fig = px.line(sales_comparison, x='Month', y='Amount($)', color='Discount_Applied',
              title='<b>Sales Amounts: With vs Without Discounts Over the Month</b>',
              labels={'Amount($)': 'Total Sales Amount ($)', 'Month': 'Month'},
              markers=True)

# Update layout to match the desired theme
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1300,
    plot_bgcolor='white',
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='Discount Applied',
        font=dict(size=12, family='Arial')
    ),
    xaxis=dict(
        title='Month',
        tickangle=-45,  # Rotate x-axis labels for readability
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        title='Total Sales Amount ($)',
        showgrid=True,
        gridcolor='lightgray',
    )
)

# Enhance markers and line visibility
fig.update_traces(
    marker=dict(size=8),  # Adjust marker size for visibility
    line=dict(width=2),  # Adjust line width for clarity
    texttemplate='%{text}',
    textposition='top center',
    textfont=dict(size=11, color='black', family='Arial', weight='bold')
)

# Show the plot
fig.show()


In [295]:
import plotly.express as px

fig = px.line(sales_comparison, x='Month', y='Amount($)', color='Discount_Applied',
              title='<b>Sales Amounts: With vs Without Discounts Over the Month</b>',
              labels={'Amount($)': 'Total Sales Amount ($)', 'Month': 'Month'},
              markers=True)

# Update layout to match the desired theme
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1000,
    plot_bgcolor='white',
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='Discount Applied',
        font=dict(size=12, family='Arial')
    ),
    xaxis=dict(
        title='Month',
        tickangle=-45,  # Rotate x-axis labels for readability
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        title='Total Sales Amount ($)',
        showgrid=True,
        gridcolor='lightgray',
    )
)

# Enhance markers and line visibility, and add black borders around markers
fig.update_traces(
    marker=dict(
        size=8,  # Adjust marker size for visibility
        line=dict(width=1.5, color='black')  # Thin black border around markers
    ),
    line=dict(width=2),  # Adjust line width for clarity
    texttemplate='%{text}',
    textposition='top center',
    textfont=dict(size=11, color='black', family='Arial', weight='bold')
)

# Show the plot
fig.show()


### 4. What are the top three cities with the highest average number of items per transaction, and how do their sales amounts vary across seasons?

In [296]:
df.head(3)

,Transaction_ID,Date,Customer_Name,Total_Items,Amount($),Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion,Month
0,1000667075,2022-09-12 17:40:23,David King,5,30.98,Debit Card,Chicago,Warehouse Club,True,Teenager,Fall,BOGO (Buy One Get One),September
1,1000156022,2022-01-20 23:03:20,Michael Williamson,3,23.29,Credit Card,Boston,Warehouse Club,True,Homemaker,Winter,Discount on Selected Items,January
2,1000681674,2022-10-15 07:49:59,Chelsea Garza,7,25.62,Debit Card,Chicago,Pharmacy,False,Teenager,Fall,Discount on Selected Items,October


In [297]:
avg_items_per_city=df.groupby("City")["Total_Items"].mean().reset_index().sort_values(by="Total_Items",ascending=False)
avg_items_per_city

,City,Total_Items
2,Chicago,5.547601
4,Houston,5.530272
6,Miami,5.521909
3,Dallas,5.489892
5,Los Angeles,5.486431
1,Boston,5.483299
7,New York,5.469845
0,Atlanta,5.461580
9,Seattle,5.457897
8,San Francisco,5.456952


In [298]:
fig = px.bar(
    avg_items_per_city,
    x="Total_Items",
    y="City",
    orientation='h',  # Horizontal bar chart
    title='<b>Average Number of Items per City</b>',
    color='City',  # Different color for each city
    color_discrete_sequence=px.colors.qualitative.Set1  # Professional color palette
)

# Update layout to match the exact theme
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1300,
    plot_bgcolor='white',
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='City',
        font=dict(size=12, family='Arial')
    ),
    xaxis=dict(
        title='Total Number of Items',
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        title='City',
        showgrid=True,
        gridcolor='lightgray',
    )
)

# Ensure the values are displayed outside the bars
fig.update_traces(
    texttemplate='%{x:.3f}',  # Show the actual number of items
    textposition='inside',  # Place text outside bars for visibility
    marker_line_width=1.5,  # Add borders around the bars for definition
    textfont=dict(size=11, color='black', family='Arial', weight='bold')
)

# Show the plot
fig.show()


In [299]:
# Get the top three cities with the highest average number of items
top_cities = avg_items_per_city.nlargest(3, 'Total_Items')
top_cities

,City,Total_Items
2,Chicago,5.547601
4,Houston,5.530272
6,Miami,5.521909


In [300]:
# Get the top three cities with the highest average number of items
top_cities = avg_items_per_city.nlargest(3, 'Total_Items')

# Filter the original dataframe for these top cities
filtered_df = df[df['City'].isin(top_cities['City'])]

# Group by City and Season to sum the sales amounts
sales_by_season = filtered_df.groupby(['City', 'Season'])['Amount($)'].sum().reset_index()

sales_by_season

,City,Season,Amount($)
0,Chicago,Fall,50343.27
1,Chicago,Spring,51831.02
2,Chicago,Summer,51261.90
3,Chicago,Winter,51710.00
4,Houston,Fall,52117.66
5,Houston,Spring,50387.61
6,Houston,Summer,50155.51
7,Houston,Winter,50564.55
8,Miami,Fall,52999.92
9,Miami,Spring,50267.66


In [301]:
fig = px.bar(
    sales_by_season,
    x='City',
    y='Amount($)',
    color='Season',  # Group bars by season
    title='<b>Sales Amounts Across Seasons for Top Cities by Average Items per Transaction</b>',
    labels={'Amount($)': 'Total Sales Amount ($)', 'City': 'City'},
    color_discrete_sequence=px.colors.qualitative.Set1,  # Professional color palette
    barmode='group'  # Side-by-side bars
)

# Update layout to match the exact theme
fig.update_layout(
    title_font=dict(size=20, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    height=600,
    width=1300,
    plot_bgcolor='white',
    margin=dict(t=60, b=100, l=50, r=50),  # Adjusted margins for better spacing
    showlegend=True,
    legend=dict(
        title='Season',
        font=dict(size=12, family='Arial')
    ),
    xaxis=dict(
        title='Total Sales Amount ($)',
        tickfont=dict(size=12, family='Arial', weight='bold'),
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        title='City',
        showgrid=True,
        gridcolor='lightgray',
    )
)

# Add borders around the bars and display values outside
fig.update_traces(
    texttemplate='%{y}',  # Display actual sales amount
    textposition='outside',  # Place text outside bars for visibility
    marker_line_width=1.5,  # Add borders around the bars
    textfont=dict(size=11, color='black', family='Arial', weight='bold')
)

# Show the plot
fig.show()


In [308]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Color palette to match your theme (using more colors for different seasons)
color_palette = px.colors.qualitative.Set1

# Sort the first chart's data in ascending order by 'Total_Items'
avg_items_per_city_sorted = avg_items_per_city.sort_values(by='Total_Items', ascending=True)

# Create subplot with 2 rows
fig_combined = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    row_heights=[0.5, 0.5],
    subplot_titles=(
        "<b>Average Number of Items per City</b>",
        "<b>Sales Amounts Across Seasons for Top Cities by Average Items per Transaction</b>"
    ),
    vertical_spacing=0.2
)

# First chart (Average Number of Items per City) sorted in ascending order
fig_combined.add_trace(
    go.Bar(
        x=avg_items_per_city_sorted['Total_Items'],
        y=avg_items_per_city_sorted['City'],
        text=avg_items_per_city_sorted['Total_Items'],
        orientation='h',
        marker_color=avg_items_per_city_sorted['City'].apply(lambda x: color_palette[avg_items_per_city_sorted['City'].tolist().index(x) % len(color_palette)]),
        texttemplate='%{text:.3f}',
        textposition='inside',
        hoverinfo='x+y+text',  # Customize hover to show x, y, and text
        showlegend=False  # Hide legend for the first plot
    ),
    row=1, col=1
)

# Second chart (Sales Amounts Across Seasons for Top Cities by Average Items per Transaction)
# Loop through unique seasons to add each season as a separate trace
for i, season in enumerate(sales_by_season['Season'].unique()):
    season_data = sales_by_season[sales_by_season['Season'] == season]
    fig_combined.add_trace(
        go.Bar(
            x=season_data['City'],
            y=season_data['Amount($)'],
            text=season_data['Amount($)'].apply(lambda x: f'{x/1000:.3f}k'),  # Format as "10k"
            texttemplate='%{text}',
            textposition='outside',
            marker_line_width=1.5,  # Add borders around the bars
            textfont=dict(size=11, color='black', family='Arial', weight='bold'),
            name=season,  # Set the name of the trace to the season
            marker_color=color_palette[i % len(color_palette)]  # Use different color for each season
        ),
        row=2, col=1
    )

# Update layout for the combined plot
fig_combined.update_layout(
    height=1000,
    width=1100,
    title_text="<b>Average Items per City & Sales Across Seasons for Top Cities</b>",
    title_font=dict(size=22, family='Arial', color='black'),
    font=dict(size=13, family='Arial'),
    plot_bgcolor='white',
    margin=dict(t=80, b=50, l=50, r=50),
    showlegend=True,
    legend=dict(
        title='Season',
        font=dict(size=12, family='Arial'),
        orientation='v',  # Vertical orientation for the legend
        yanchor='auto',
        y=.15,
        x=1.05
    ),
    barmode='group',  # Ensure side-by-side bars for the second chart
)

# Customize x-axis and y-axis titles and style for both plots
fig_combined.update_xaxes(
    title_text="Total Number of Items",
    row=1, col=1,
    showgrid=True,
    gridcolor='lightgray'
)
fig_combined.update_yaxes(
    title_text="City",
    row=1, col=1,
    showgrid=True,
    gridcolor='lightgray'
)

# Update x-axis and y-axis for the second plot (Sales Amounts)
fig_combined.update_xaxes(
    title_text="Total Sales Amount ($)",
    row=2, col=1,
    showgrid=True,
    gridcolor='lightgray'
)
fig_combined.update_yaxes(
    title_text="City",
    row=2, col=1,
    showgrid=True,
    gridcolor='lightgray'
)

# Show the combined plot
fig_combined.show()
